In [1]:
import os
import contextlib

import pandas as pd
import seaborn as sns
from datetime import date
import plotly.express as px
import plotly.graph_objects as go

In [2]:
def find_sheet_name(sheet_names):
    cpi_sheet_names = list(filter(lambda x: "fdi" in x.lower(), sheet_names))

    if len(cpi_sheet_names) == 1:
        return cpi_sheet_names[0]

    cpi_sheet_names = list(filter(lambda x: "dtnn" in x.lower(), sheet_names))

    if len(cpi_sheet_names) == 1:
        return cpi_sheet_names[0]

    if len(sheet_names) == 43:
        return "22"

    cpi_sheet_names = list(filter(lambda x: "sheet1" in x.lower(), sheet_names))

    if len(cpi_sheet_names) == 1:
        return cpi_sheet_names[0]

    raise RuntimeError(sheet_names, len(sheet_names))

In [3]:
en_to_vn = {
    "TOTAL": "TỔNG SỐ",
    "By province": "Phân theo một số địa phương",
    "Bac Lieu": "Bạc Liêu",
    "Hai Phong": "Hải Phòng",
    "Binh Duong": "Bình Dương",
    "Ha Nam": "Hà Nam",
    "Ha Noi": "Hà Nội",
    "Ho Chi Minh city": "TP. Hồ Chí Minh",
    "Tay Ninh": "Tây Ninh",
    "Bac Ninh": "Bắc Ninh",
    "Quang Ninh": "Quảng Ninh",
    "Bac Giang": "Bác Giang",
    "Quang Binh": "Quảng Bình",
    "Ba Ria - Vung Tau": "Bà Rịa - Vũng Tàu",
    "Long An": "Long An",
    "Thanh Hoa": "Thanh Hóa",
    "Dong Nai": "Đồng Nai",
    "Phu Tho": "Phú Thọ",
    "Da Nang": "Đà Nẵng",
    "Hai Duong": "Hải Dương",
    "Vinh Phuc": "Vĩnh Phúc",
    "Vinh Long": "Vĩnh Long",
    "By country and geographical territory": "Phân theo một số nước và vùng lãnh thổ",
    "Singapore": "Xin-ga-po",
    "China": "Trung Quốc",
    "South Korea": "Hàn Quốc",
    "Special Administration Hong Kong": "Đặc khu HC Hồng Công (TQ)",
    "Taiwan": "Đài Loan",
    "Japan": "Nhật Bản",
    "Philippines": "Phi-li-pin",
    "BritishVirginIslands": "Quần đảo Virgin thuộc Anh",
    "Thailand": "Thái Lan",
    "Netherlands": "Hà Lan",
    "Cayman Islands": "Quần đảo Cay-man",
    "Samoa": "Xa-moa",
    "United State": "Hoa Kỳ",
    "Seychelles": "Xây-Sen",
    "Malaysia": "Ma-lai-xi-a",
    "France": "Pháp",
    "Germany": "Đức",
}

In [4]:
def get_cpi(xl, sheet_name, date):
    df = xl.parse(sheet_name)

    if (
        len(
            df[df.columns[1]][
                (df[df.columns[1]] == "Phân theo một số địa phương")
                | (df[df.columns[1]] == "By province")
            ].index
        )
        > 0
    ):
        df[df.columns[1]] = df[df.columns[1]].fillna(df[df.columns[0]])
        df = df.drop(columns=[df.columns[0]])

    if (
        len(
            df[df.columns[1]][
                (df[df.columns[1]] == "TP. Hồ Chí Minh")
                | (df[df.columns[1]] == "Ho Chi Minh city")
                | (df[df.columns[1]] == " TP. Hồ Chí Minh ")
            ].index
        )
        > 0
    ):
        df[df.columns[1]] = df[df.columns[1]].fillna(df[df.columns[0]])
        df = df.drop(columns=[df.columns[0]])

    df[df.columns[0]] = (
        df[df.columns[0]].replace(r"\s+", " ", regex=True).replace(r"\n", " ", regex=True)
    )

    df[df.columns[0]] = df[df.columns[0]].str.strip()

    df = df.drop(
        list(
            range(
                df[df.columns[0]][
                    (df[df.columns[0]] == "TỔNG SỐ") | ((df[df.columns[0]] == "TOTAL"))
                ].index[0]
            )
        )
    )

    if len(df[df.columns[0]][df[df.columns[0]] == "TOTAL"].index) > 0:
        df[df.columns[0]] = df[df.columns[0]].map(en_to_vn)

    if len(df.columns) == 4:
        df.columns = [
            "Phân Loại",
            "Số dự án",
            "Vốn đăng ký cấp mới",
            "Vốn đăng ký điều chỉnh",
        ]

        df = df[pd.to_numeric(df["Số dự án"], errors="coerce").notnull()]
        df = df[pd.to_numeric(df["Vốn đăng ký cấp mới"], errors="coerce").notnull()]
        df = df[pd.to_numeric(df["Vốn đăng ký điều chỉnh"], errors="coerce").notnull()]

        df["Vốn đăng ký"] = df["Vốn đăng ký cấp mới"] + df["Vốn đăng ký điều chỉnh"]
    else:
        df.columns = [
            "Phân Loại",
            "Số dự án",
            "Vốn đăng ký",
        ]

        df = df[pd.to_numeric(df["Số dự án"], errors="coerce").notnull()]
        df = df[pd.to_numeric(df["Vốn đăng ký"], errors="coerce").notnull()]

    df["Date"] = date
    df["Date"] = pd.to_datetime(df["Date"], dayfirst=False)

    return df


xl = pd.ExcelFile("datas/2021-12-29-03-Bieu-2021-28.12.xlsx")

sheet_name = find_sheet_name(xl.sheet_names)

get_cpi(xl, sheet_name, "2021-12-29")

,Phân Loại,Số dự án,Vốn đăng ký cấp mới,Vốn đăng ký điều chỉnh,Vốn đăng ký,Date
5,TỔNG SỐ,1738,15245.4,9014.8,24260.2,2021-12-29
7,Long An,54,3518.8,255.2,3774.0,2021-12-29
8,Cần Thơ,5,1316.8,9,1325.8,2021-12-29
9,Bắc Ninh,126,1170.5,320.6,1491.1,2021-12-29
10,Quảng Ninh,10,1011.5,114,1125.5,2021-12-29
11,Vĩnh Phúc,27,703.1,33.2,736.3,2021-12-29
12,TP. Hồ Chí Minh,633,686.6,1124.3,1810.9,2021-12-29
13,Bình Dương,75,629.6,669.5,1299.1,2021-12-29
14,Bắc Giang,21,621.8,652.7,1274.5,2021-12-29
15,Thái Bình,6,459.9,1.4,461.3,2021-12-29


In [5]:
import os
import contextlib

dfs_fdi_origin = []
for file_name in sorted(
    os.listdir("./datas"),
    key=lambda filename: f"{filename.split('-')[0]}-{filename.split('-')[1]}-{filename.split('-')[2]}",
    reverse=True,
):
    if file_name.startswith(".~"):
        continue

    try:
        file_path = os.path.join("./datas", file_name)

        year = int(file_name.split("-")[0])
        month = int(file_name.split("-")[1])
        day = int(file_name.split("-")[2])

        if year < 2016:
            continue

        xl = pd.ExcelFile(file_path)

        df_fdi = get_cpi(xl, find_sheet_name(xl.sheet_names), f"{year}-{month}-{day}")

        dfs_fdi_origin.append(df_fdi)
    except Exception as e:
        print(file_name)
        raise e

In [ ]:
dfs_fdi = pd.concat(dfs_fdi_origin, ignore_index=True, axis=0)
dfs_fdi["Date"] = pd.to_datetime(dfs_fdi["Date"], dayfirst=False)
dfs_fdi

,Phân Loại,Số dự án,Vốn đăng ký cấp mới,Vốn đăng ký điều chỉnh,Vốn đăng ký,Date
0,TỔNG SỐ,2254,10233.343473,5150.443695,15383.787168,2023-09-29
1,Bắc Giang,72,1412.98117,321.942192,1734.923362,2023-09-29
2,Nghệ An,14,1015.933632,256.78996,1272.723592,2023-09-29
3,Bắc Ninh,266,852.306697,378.744122,1231.050819,2023-09-29
4,Bình Phước,35,689.896028,-2.684363,687.211665,2023-09-29
...,...,...,...,...,...,...
3274,Quần đảo Vigin thuộc Anh,3,NaN,NaN,9.4,2016-01-29
3275,Đài Loan,6,NaN,NaN,8.7,2016-01-29
3276,Thái Lan,3,NaN,NaN,6.3,2016-01-29
3277,Hà Lan,1,NaN,NaN,6,2016-01-29


In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

df_show = dfs_fdi[(dfs_fdi["Phân Loại"] == "TỔNG SỐ")]

fig = px.area(df_show, x="Date", y="Vốn đăng ký", color="Phân Loại")

fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=[
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(count=3, label="3y", step="year", stepmode="backward"),
            dict(count=5, label="5y", step="year", stepmode="backward"),
            dict(count=7, label="7y", step="year", stepmode="backward"),
            dict(count=10, label="10y", step="year", stepmode="backward"),
            dict(step="all"),
        ]
    ),
)

fig.show()

/home/hien/.pyenv/versions/3.9.16/envs/vn-macroeconomics/lib/python3.9/site-packages/_plotly_utils/basevalidators.py:105: FutureWarning: The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result
  v = v.dt.to_pydatetime()
